---
toc: false
comments: true
layout: post
title: What is CRUD?
description: Understanding CRUD
type: hacks
courses: { compsci: {week: 19} }
---

# CRUD

## What is CRUD?

CRUD stands for:

C-reate: Involves the creation of addition of data to a database. In essence, it "create"s entries in databases.

R-ead: Involves the retrieval, querying, or "read"ing of existing data from a database.

U-pdate: Involves the modification or "update"ing of existing data in a database. Operation of changing the values of specific fields in a system. 

D-elete: Involves the removal or "delete"ion of existing data from a database. It's the operation of eliminating information from a database.

In [ ]:
import json
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

from model.users import User

user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(user_api)

class UserAPI:        
    class _CRUD(Resource):  # User API operation for Create, Read.  THe Update, Delete methods need to be implemeented
        def post(self): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password
            password = body.get('password')

            active_classes = body.get('active_classes')
            if active_classes is None or len(active_classes) == 0:
                return {'message': 'Active class is missing'}, 400

            archived_classes = body.get('archived_classes')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid,
                      active_classes=active_classes,
                      archived_classes=archived_classes)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # set server request status    
            uo.server_needed = body.get('server_needed')
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        def get(self): # Read Method
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps
        
        def put(self, id):  # Update method
            ''' Read data for json body '''
            body = request.get_json()

            ''' Find user by ID '''
            user = User.query.get(id)
            if user is None:
                return {'message': f'User with ID {id} not found'}, 404

            ''' Update fields '''
            name = body.get('name')
            if name:
                user.name = name
                
            uid = body.get('uid')
            if uid:  
                user.uid = uid
                
            user.server_needed = body.get('server_needed')

            user.active_classes = body.get('active_classes')
            user.archived_classes = body.get('archived_classes')

            ''' Commit changes to the database '''
            user.update()
            return jsonify(user.read())
        
        def delete(self, id):  # Delete method
            ''' Find user by ID '''
            user = User.query.get(id)
            if user is None:
                return {'message': f'User with ID {id} not found'}, 404

            ''' Delete user '''
            user.delete()
            return {'message': f'User with ID {id} has been deleted'}

    
    class _Security(Resource):

        def post(self):
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Get Data '''
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            password = body.get('password')
            
            ''' Find user '''
            user = User.query.filter_by(_uid=uid).first()
            if user is None or not user.is_password(password):
                return {'message': f"Invalid user id or password"}, 400
            
            ''' authenticated user '''
            return jsonify(user.read())

            
    # building RESTapi endpoint
    api.add_resource(_CRUD, '/', '/<int:id>')
    api.add_resource(_Security, '/authenticate')
    